In [1]:
import random
import torch
from d2l import torch as d2l

In [64]:
def seq_data_iter_random(corpus, batch_size, num_steps):  #@save
    """使用随机抽样生成一个小批量子序列"""
    # 从随机偏移量开始对序列进行分区，随机范围包括num_steps-1
    corpus = corpus[random.randint(0, num_steps - 1):]
    # 减去1，是因为我们需要考虑标签
    num_subseqs = (len(corpus) - 1) // num_steps
    # 长度为num_steps的子序列的起始索引
    initial_indices = list(range(0, num_subseqs * num_steps, num_steps))
    # 在随机抽样的迭代过程中，
    # 来自两个相邻的、随机的、小批量中的子序列不一定在原始序列上相邻
    random.shuffle(initial_indices)

    def data(pos):
        # 返回从pos位置开始的长度为num_steps的序列
        return corpus[pos: pos + num_steps]

    num_batches = num_subseqs // batch_size
    for i in range(0, batch_size * num_batches, batch_size):
        # 在这里，initial_indices包含子序列的随机起始索引
        initial_indices_per_batch = initial_indices[i: i + batch_size]
        X = [data(j) for j in initial_indices_per_batch]
        Y = [data(j + 1) for j in initial_indices_per_batch]
        yield torch.tensor(X), torch.tensor(Y)

In [84]:
my_seq = list(range(32))
for X, Y in seq_data_iter_random(my_seq, batch_size=2, num_steps=5):
    print('X: ', X, '\nY:', Y)

X:  tensor([[21, 22, 23, 24, 25],
        [11, 12, 13, 14, 15]]) 
Y: tensor([[22, 23, 24, 25, 26],
        [12, 13, 14, 15, 16]])
X:  tensor([[16, 17, 18, 19, 20],
        [26, 27, 28, 29, 30]]) 
Y: tensor([[17, 18, 19, 20, 21],
        [27, 28, 29, 30, 31]])
X:  tensor([[ 6,  7,  8,  9, 10],
        [ 1,  2,  3,  4,  5]]) 
Y: tensor([[ 7,  8,  9, 10, 11],
        [ 2,  3,  4,  5,  6]])


In [ ]:
def seq_data_iter_sequential(corpus, batch_size, num_steps):  #@save
    """使用顺序分区生成一个小批量子序列"""
    # 从随机偏移量开始划分序列
    offset = random.randint(0, num_steps)
    num_tokens = ((len(corpus) - offset - 1) // batch_size) * batch_size
    Xs = torch.tensor(corpus[offset: offset + num_tokens])
    Ys = torch.tensor(corpus[offset + 1: offset + 1 + num_tokens])
    Xs, Ys = Xs.reshape(batch_size, -1), Ys.reshape(batch_size, -1)
    num_batches = Xs.shape[1] // num_steps
    for i in range(0, num_steps * num_batches, num_steps):
        X = Xs[:, i: i + num_steps]
        Y = Ys[:, i: i + num_steps]
        yield X, Y

In [97]:
num_steps=5
corpus = list(range(32))
offset = random.randint(0, num_steps)
batch_size = 2

In [98]:
num_tokens = ((len(corpus) - offset - 1) // batch_size) * batch_size


In [99]:
Xs = torch.tensor(corpus[offset: offset + num_tokens])


In [100]:
Xs

tensor([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
        20, 21, 22, 23, 24, 25, 26, 27, 28, 29])

In [101]:
Ys = torch.tensor(corpus[offset + 1: offset + 1 + num_tokens])

In [102]:
Ys

tensor([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
        21, 22, 23, 24, 25, 26, 27, 28, 29, 30])

In [103]:
Xs, Ys = Xs.reshape(batch_size, -1), Ys.reshape(batch_size, -1)

In [104]:
Xs.shape

torch.Size([2, 14])

In [105]:
num_batches = Xs.shape[1] // num_steps


In [106]:
Xs

tensor([[ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
        [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]])

In [107]:
Ys

tensor([[ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16],
        [17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]])

In [109]:
for i in range(0, num_steps * num_batches, num_steps):
        X = Xs[:, i: i + num_steps]
        Y = Ys[:, i: i + num_steps]
        print(X, Y)

tensor([[ 2,  3,  4,  5,  6],
        [16, 17, 18, 19, 20]]) tensor([[ 3,  4,  5,  6,  7],
        [17, 18, 19, 20, 21]])
tensor([[ 7,  8,  9, 10, 11],
        [21, 22, 23, 24, 25]]) tensor([[ 8,  9, 10, 11, 12],
        [22, 23, 24, 25, 26]])


In [ ]:
class SeqDataLoader:  #@save
    """加载序列数据的迭代器"""
    def __init__(self, batch_size, num_steps, use_random_iter, max_tokens):
        if use_random_iter:
            self.data_iter_fn = d2l.seq_data_iter_random
        else:
            self.data_iter_fn = d2l.seq_data_iter_sequential
        self.corpus, self.vocab = d2l.load_corpus_time_machine(max_tokens)
        self.batch_size, self.num_steps = batch_size, num_steps

    def __iter__(self):
        return self.data_iter_fn(self.corpus, self.batch_size, self.num_steps)

In [111]:
ll = ['<unk>', '<pad>', '<bos>', '<eos>', '的', '，', '在', '。', '经', '之', '\n', '其', '后', '大', '已', '朴']
t1 = torch.tensor([2,3,4])

In [1]:
import torch
from d2l import torch as d2l

X, W_xh = torch.normal(0, 1, (3, 1)), torch.normal(0, 1, (1, 4))
H, W_hh = torch.normal(0, 1, (3, 4)), torch.normal(0, 1, (4, 4))
torch.matmul(X, W_xh) + torch.matmul(H, W_hh)

tensor([[ 0.9736, -0.3140,  0.5824,  1.7299],
        [-2.4450,  0.9621,  0.8928, -1.9603],
        [-0.2907,  2.3265, -0.7343, -3.4360]])

In [4]:
torch.mm(torch.cat((X,H),dim=1),torch.cat((W_xh,W_hh),dim=0))

tensor([[ 0.9736, -0.3140,  0.5824,  1.7299],
        [-2.4450,  0.9621,  0.8928, -1.9603],
        [-0.2907,  2.3265, -0.7343, -3.4360]])